In [121]:
!pip install transformers==4.11.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 27.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 65.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-310/t

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

import os

from utils import *

from answer_generation import __load_experiment, get_tokenizer, get_model

### load model

In [2]:
base_model = 'potsawee/t5-large-generation-squad-QuestionAnswer'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-squad-QuestionAnswer")

In [4]:
processer = get_tokenizer(base_model)
model = get_model(base_model, device, processer)

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at potsawee/t5-large-generation-squad-QuestionAnswer and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model_dir = 't5-l-SQuAD_e20_lr5e-06_eps5e-06_Adam_cosine_batch1'

In [6]:
if os.path.exists(os.path.join(model_dir, 'train.log')):
    # get current epoch
    current_epoch = 0
    with open(os.path.join(model_dir, 'train.log')) as f:
        for line in f:
            current_epoch += 1
    # get the latest model
    state_dict = torch.load(os.path.join(model_dir, 'latest_model.pt'), map_location=device.type)
    model.load_state_dict(state_dict['weights'])
else: 
    print("model not found")

RuntimeError: Error(s) in loading state_dict for T5ForQuestionAnswering:
	Missing key(s) in state_dict: "qa_outputs.weight", "qa_outputs.bias". 
	Unexpected key(s) in state_dict: "lm_head.weight". 
	size mismatch for shared.weight: copying a param with shape torch.Size([32101, 1024]) from checkpoint, the shape in current model is torch.Size([32128, 1024]).
	size mismatch for encoder.embed_tokens.weight: copying a param with shape torch.Size([32101, 1024]) from checkpoint, the shape in current model is torch.Size([32128, 1024]).
	size mismatch for decoder.embed_tokens.weight: copying a param with shape torch.Size([32101, 1024]) from checkpoint, the shape in current model is torch.Size([32128, 1024]).

### load data

In [7]:
data_path = 'Question_Answer_Dataset_v1.2'
feature_cache_path = 'Question_Answer_Dataset_v1.2/features_answers'

# prepare feature data if not yet exist 
if not (os.path.exists(feature_cache_path) and os.path.isfile(feature_cache_path)):
    # use the encoder to get the raw dataset (context are extracted by IR with the K-NN sentence to the QA pair)
    print("processing raw dataset... ")
    raw_dataset = CustomData(data_path, encoder_model, k=1)
    print("computing features...")
    # tokenize
    prepare_features_a(raw_dataset, feature_cache_path, processer, max_len_inp=512,max_len_out=512)
else:
    print("features exists")

# feature dataset
test_dataset = FeatureData(feature_cache_path, 'test', 425) 

features exists
length of feature test set:  425


In [7]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32101, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32101, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [4]:
context = r"""Giraffes can inhabit savannas, grasslands, or open woodlands. They prefer areas enriched with acacia growth. They drink large quantities of water and, as a result, they can spend long periods of time in dry, arid areas. When searching for more food they will venture into areas with denser foliage.""".replace('\n', ' ')

In [11]:
inputs = processer(context, return_tensors='pt')

In [12]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

In [13]:
input_ids = inputs['input_ids'].to(device)

In [14]:
outputs = model.generate(input_ids)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [15]:
question_answer = processer.decode(outputs[0], skip_special_tokens=False)
question_answer = question_answer.replace(processer.pad_token, "").replace(processer.eos_token, "")
question, answer = question_answer.split(processer.sep_token)

In [5]:
question = 'What type of vegetation do giraffes prefer?'

In [6]:
answer = 'acacia'

### answering

In [7]:
question_answerer = pipeline("question-answering", model='t5-base')

Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at t5-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` 

In [4]:
question_answerer.model

T5ForQuestionAnswering(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout

In [8]:
question

'What type of vegetation do giraffes prefer?'

In [9]:
context

'Giraffes can inhabit savannas, grasslands, or open woodlands. They prefer areas enriched with acacia growth. They drink large quantities of water and, as a result, they can spend long periods of time in dry, arid areas. When searching for more food they will venture into areas with denser foliage.'

In [10]:
question_answerer(question=question, context=context)

{'score': 0.000419730378780514,
 'start': 0,
 'end': 30,
 'answer': 'Giraffes can inhabit savannas,'}

In [13]:
question = "Is the bear white?"
context = "the bear is white"

In [16]:
inputs = f"question: {question} context: {context}"
inputs

'question: Is the bear white? context: the bear is white'

In [17]:
question_answerer(question=question, context=context)

{'score': 0.0747537910938263, 'start': 0, 'end': 8, 'answer': 'the bear'}